In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('/content/drive/MyDrive/Depression detection dataset/BSMDD_main.xlsx')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21910 entries, 0 to 21909
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_bengali   21910 non-null  object
 1   text_banglish  21910 non-null  object
 2   label          21910 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 513.6+ KB


In [ ]:
df.head()

,text_bengali,text_banglish,label
0,মানসিক শারীরিকভাবে অসুস্থ ক্লান্ত পুরো জীবন শা...,manasika sharirikabhabe asustha klanta puro ji...,1
1,দয়া সাথে থাকুন অত্যন্ত দীর্ঘ আপনাকে পড়তে উত্...,daya sathe thakuna atyanta dirgha apanake pada...,1
2,জানতাম সাথে ভুল লোক খারাপ জীবন কাটিয়েছে সম্পূ...,janatama sathe bhula loka kharapa jibana katiy...,1
3,অনেটিভ ইংরেজি স্পিকারের অনুসরণ বিরক্তিকর অপ্রত...,anetibha imreji spikarera anusarana biraktikar...,1
4,অনেটিভ ইংরেজি স্পিকারের অনুসরণ বিরক্তিকর অপ্রত...,anetibha imreji spikarera anusarana biraktikar...,1


In [2]:
! pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
openc

In [7]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# For complexity analysis
import time
import os

# The correct resource name is 'punkt'
nltk.download('punkt')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# 1. Load the dataset
file_path = '/content/drive/MyDrive/Depression detection dataset/BSMDD_main.xlsx'
df = pd.read_excel(file_path)

# Drop any rows with missing text
df.dropna(subset=['text_banglish'], inplace=True)
df.reset_index(drop=True, inplace=True)


# 2. Define a text preprocessing function
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase, ensure it's a string
    text = re.sub(r'[^a-z\s]', '', text) # Remove punctuation, numbers, and special characters
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

# 3. Apply the preprocessing function
df['cleaned_text'] = df['text_banglish'].apply(preprocess_text)

# 4. Tokenize the text using a simple split
df['tokenized_text'] = df['cleaned_text'].apply(lambda x: x.split())

# Display the first few rows with the new columns
print("DataFrame after preprocessing and tokenization:")
# CORRECTED: Changed 'Label' to 'label' (all lowercase)
print(df[['text_banglish', 'cleaned_text', 'tokenized_text', 'label']].head())

# Display dataset info to check for null values
print("\nDataset Info:")
df.info()

DataFrame after preprocessing and tokenization:
                                       text_banglish  \
0  manasika sharirikabhabe asustha klanta puro ji...   
1  daya sathe thakuna atyanta dirgha apanake pada...   
2  janatama sathe bhula loka kharapa jibana katiy...   
3  anetibha imreji spikarera anusarana biraktikar...   
4  anetibha imreji spikarera anusarana biraktikar...   

                                        cleaned_text  \
0  manasika sharirikabhabe asustha klanta puro ji...   
1  daya sathe thakuna atyanta dirgha apanake pada...   
2  janatama sathe bhula loka kharapa jibana katiy...   
3  anetibha imreji spikarera anusarana biraktikar...   
4  anetibha imreji spikarera anusarana biraktikar...   

                                      tokenized_text  label  
0  [manasika, sharirikabhabe, asustha, klanta, pu...      1  
1  [daya, sathe, thakuna, atyanta, dirgha, apanak...      1  
2  [janatama, sathe, bhula, loka, kharapa, jibana...      1  
3  [anetibha, imreji, spikarer

In [18]:
from gensim.models import Word2Vec

# Isolate the tokenized sentences for training
sentences = df['tokenized_text'].tolist()

# --- Word2Vec Model Training ---
# Define the model parameters
embedding_dim = 100  # Dimensionality of the word vectors
window_size = 5      # Context window size
min_word_count = 1   # Minimum word count to be included

# sg=1 trains the Skip-Gram model, which often works well
w2v_model = Word2Vec(sentences=sentences,
                     vector_size=embedding_dim,
                     window=window_size,
                     min_count=min_word_count,
                     workers=4,
                     sg=1)

print("Word2Vec model trained successfully.")

# --- Sanity Check ---
# Check the vocabulary size
vocab_size = len(w2v_model.wv.index_to_key)
print(f"\nVocabulary size: {vocab_size}")

# Test the model by finding the most similar words to a sample word.
# 'ami' ('I' in Bengali) is a common word and likely to be in the vocabulary.
try:
    if 'ami' in w2v_model.wv:
        sample_word = 'ami'
    else:
        # If 'ami' isn't in the vocab, just use the most frequent word as an example
        sample_word = w2v_model.wv.index_to_key[0]

    similar_words = w2v_model.wv.most_similar(sample_word)
    print(f"\nWords most similar to '{sample_word}':")
    print(similar_words)
except (IndexError, KeyError):
    print("\nCould not perform similarity check. The vocabulary might be smaller than expected.")

Word2Vec model trained successfully.

Vocabulary size: 56826

Words most similar to 'ami':
[('prototaipa', 0.9431474804878235), ('merinase', 0.9421924948692322), ('bhayesachyatera', 0.9418488144874573), ('iujaranema', 0.9399396777153015), ('resalimye', 0.9384870529174805), ('lukochurira', 0.9363691806793213), ('anugamidera', 0.9354920983314514), ('trata', 0.9353041052818298), ('alishake', 0.9349705576896667), ('hostim', 0.9338364005088806)]


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

# --- 1. Use Keras Tokenizer to convert texts to integer sequences ---
# We use the cleaned text column for this
texts = df['cleaned_text'].tolist()
labels = df['label'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
# The Keras vocab size is the number of unique words + 1 (for the padding token)
keras_vocab_size = len(word_index) + 1
print(f"Vocabulary size for Keras Tokenizer: {keras_vocab_size}")


# --- 2. Pad sequences to a uniform length ---
# Let's find a suitable max length (e.g., the 95th percentile of sentence lengths)
# This avoids making all sequences excessively long because of a few outliers.
lengths = [len(s) for s in sequences]
maxlen = int(np.percentile(lengths, 95))
print(f"Padding sequences to a max length of: {maxlen}")

X = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
y = labels


# --- 3. Create the embedding matrix from our Word2Vec model ---
embedding_matrix = np.zeros((keras_vocab_size, embedding_dim)) # embedding_dim is 100 from the previous cell

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_vector = w2v_model.wv[word]
        embedding_matrix[i] = embedding_vector

print(f"Shape of the embedding matrix: {embedding_matrix.shape}")


# --- 4. Split data into training and testing sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y) # stratify ensures similar class distribution

print(f"\nTraining data shape (X_train): {X_train.shape}")
print(f"Testing data shape (X_test): {X_test.shape}")

Vocabulary size for Keras Tokenizer: 56827
Padding sequences to a max length of: 192
Shape of the embedding matrix: (56827, 100)

Training data shape (X_train): (17528, 192)
Testing data shape (X_test): (4382, 192)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall

# --- 1. Build the BiGRU Model ---

model = Sequential()

# Embedding Layer (loaded with our Word2Vec weights)
model.add(Embedding(input_dim=keras_vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False)) # Freeze the embedding layer

# Bidirectional GRU Layer
model.add(Bidirectional(GRU(units=64)))

# Dropout for regularization
model.add(Dropout(0.3))

# A standard Dense layer
model.add(Dense(32, activation='relu'))

# Final Output Layer for binary classification
model.add(Dense(1, activation='sigmoid'))


# --- 2. Compile the Model ---
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])


# --- 3. Print Model Summary ---
# This is very useful for your paper to show the model's structure and parameters.
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     5,682,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,682,700 (21.68 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 5,682,700 (21.68 MB)

In [15]:
import time

# --- Train the Model ---

print("Starting model training...")

# Add a callback for Early Stopping
# This will stop training if the validation loss doesn't improve for 3 consecutive epochs.
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define number of epochs and batch size
epochs = 20
batch_size = 32

# Start the timer
start_time = time.time()

# Train the model
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])

# Stop the timer
end_time = time.time()

# Calculate and print the training time
training_time = end_time - start_time
print(f"\nTraining finished in {training_time:.2f} seconds.")

Starting model training...
Epoch 1/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 139s 241ms/step - accuracy: 0.7767 - loss: 0.4715 - precision: 0.7556 - recall: 0.8395 - val_accuracy: 0.8665 - val_loss: 0.3191 - val_precision: 0.8231 - val_recall: 0.9339
Epoch 2/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 134s 245ms/step - accuracy: 0.8724 - loss: 0.3070 - precision: 0.8576 - recall: 0.8919 - val_accuracy: 0.8884 - val_loss: 0.2746 - val_precision: 0.9018 - val_recall: 0.8718
Epoch 3/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 146s 266ms/step - accuracy: 0.8933 - loss: 0.2620 - precision: 0.8837 - recall: 0.9061 - val_accuracy: 0.8982 - val_loss: 0.2487 - val_precision: 0.8747 - val_recall: 0.9297
Epoch 4/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 141s 257ms/step - accuracy: 0.9013 - loss: 0.2374 - precision: 0.8931 - recall: 0.9115 - val_accuracy: 0.9019 - val_loss: 0.2370 - val_precision: 0.8804 - val_recall: 0.9302
Epoch 5/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 138s 251ms/step - accuracy: 0.9107 - loss: 0.2235 - precision: 0.9043 - recal

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import time
import os
import numpy as np

# --- 1. Performance Evaluation on Test Set ---
print("--- Final Performance Evaluation ---")

# Get model's prediction probabilities on the test set
y_pred_probs = model.predict(X_test)

# Convert probabilities to binary class labels (0 or 1) using a 0.5 threshold
y_pred = (y_pred_probs > 0.5).astype("int32")

# Calculate and print the final metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\nOverall Test Metrics:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}\n")

# Display a detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Depressed (0)', 'Depressed (1)']))

# Display the confusion matrix
print("Confusion Matrix:")
# [[True Negative, False Positive], [False Negative, True Positive]]
print(confusion_matrix(y_test, y_pred))
print("\n" + "="*50 + "\n")


# --- 2. Practical Time Complexity Analysis ---
print("--- Time Complexity ---")

# We already have the training time from the previous step
print(f"Total Training Time: {training_time:.2f} seconds (approx. {training_time/60:.1f} minutes)")

# Measure Inference Time on the entire test set
inference_start_time = time.time()
_ = model.predict(X_test)
inference_end_time = time.time()

total_inference_time = inference_end_time - inference_start_time
avg_inference_time_per_sample = total_inference_time / len(X_test)

print(f"Total Inference Time for {len(X_test)} samples: {total_inference_time:.4f} seconds")
print(f"Average Inference Time per Sample: {avg_inference_time_per_sample * 1000:.4f} milliseconds\n")
print("="*50 + "\n")


# --- 3. Practical Space Complexity Analysis ---
print("--- Space Complexity ---")

# Number of trainable parameters in the model
total_params = model.count_params()
print(f"Total Model Parameters: {total_params:,}")

# Save the model to a file to check its size
model_filename = "depression_detection_bigru.keras"
model.save(model_filename)
model_size_bytes = os.path.getsize(model_filename)
model_size_mb = model_size_bytes / (1024 * 1024)

print(f"Model Size on Disk: {model_size_mb:.2f} MB")

--- Final Performance Evaluation ---
137/137 ━━━━━━━━━━━━━━━━━━━━ 15s 98ms/step

Overall Test Metrics:
Accuracy:  0.9133
Precision: 0.9023
Recall:    0.9270
F1 Score:  0.9145

Classification Report:
                   precision    recall  f1-score   support

Not Depressed (0)       0.92      0.90      0.91      2190
    Depressed (1)       0.90      0.93      0.91      2192

         accuracy                           0.91      4382
        macro avg       0.91      0.91      0.91      4382
     weighted avg       0.91      0.91      0.91      4382

Confusion Matrix:
[[1970  220]
 [ 160 2032]]


--- Time Complexity ---
Total Training Time: 1683.59 seconds (approx. 28.1 minutes)
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step
Total Inference Time for 4382 samples: 6.9408 seconds
Average Inference Time per Sample: 1.5839 milliseconds


--- Space Complexity ---
Total Model Parameters: 5,750,605
Model Size on Disk: 22.50 MB


In [17]:
import pickle

# Save the tokenizer object
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Tokenizer saved to 'tokenizer.pickle'")

Tokenizer saved to 'tokenizer.pickle'
